In [ ]:
!pip install pymongo
!pip install google-cloud-storage
!pip install fuzzywuzzy
!pip install "pymongo[srv]"
!pip install  azure-storage-blob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.6/408.6 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 15.3 MB/s eta 0:00:00


In [ ]:
from google.cloud import storage
import pandas as pd
import pandas as pd
import numpy as np
import requests
import json
from io import StringIO
from pymongo import MongoClient
from fuzzywuzzy import fuzz
import math
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
from pymongo.errors import BulkWriteError
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from datetime import date
import time

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
# Specify the name of your bucket
AZURE_STORAGE_CONTAINER = "yelp"
AZURE_CONNECTION_STRING = ""
raw_folder_name = "raw"
blob_name = f"{raw_folder_name}/restaurants.csv"


In [ ]:
def fetch_api_data(url, headers):
    """Fetch data from the API."""
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise an error for bad status codes
    return response.json()  # Return the response as JSON

def upload_to_blob(connection_string, container_name, blob_name, data):
    """Upload data to Azure Blob Storage."""
    # Initialize BlobServiceClient
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)

    # Get the container client
    container_client = blob_service_client.get_container_client(container_name)

    # Ensure the container exists
    try:
        container_client.create_container()
    except Exception:
        pass  # Ignore if the container already exists

    # Get the blob client
    blob_client = container_client.get_blob_client(blob_name)

    # Upload data
    blob_client.upload_blob(data, overwrite=True)
    print(f"Data successfully uploaded to {blob_name} in container {container_name}")

def download_csv_from_blob(connection_string, container_name, blob_name):
    """Download a CSV file from Azure Blob Storage and return it as a DataFrame."""
    # Initialize BlobServiceClient
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)

    # Get the blob client
    blob_client = blob_service_client.get_blob_client(container_name, blob_name)

    # Download the blob content
    print(f"Downloading {blob_name} from Azure Blob Storage...")
    download_stream = blob_client.download_blob()

    # Convert to a Pandas DataFrame
    csv_data = download_stream.content_as_text()
    data_frame = pd.read_csv(csv_data)
    print(f"Successfully downloaded {blob_name} and converted to DataFrame.")
    return data_frame

In [ ]:
# API Configuration
API_URL = "https://data.cityofnewyork.us/api/views/43nn-pn8j/rows.csv?fourfour=43nn-pn8j&cacheBust=1732217650&date=20241121&accessType=DOWNLOAD"
HEADERS = {
    "Authorization": "Bearer your_api_token",
    "Accept": "application/json",
}

def main():
    try:
        # Fetch data from the API
        print("Fetching data from the API...")
        api_data = fetch_api_data(API_URL, HEADERS)

        # Convert JSON data to a string
        api_data_str = str(api_data)

        # Upload the data to Azure Blob Storage
        print("Uploading data to Azure Blob Storage...")
        upload_to_blob(AZURE_CONNECTION_STRING, AZURE_STORAGE_CONTAINER, blob_name, api_data_str)

        # Download a CSV file from Blob Storage
        print("Retrieving CSV data from Azure Blob Storage...")
        df = download_csv_from_blob(AZURE_CONNECTION_STRING, AZURE_STORAGE_CONTAINER, blob_name)

        # Display the first few rows of the DataFrame
        print("CSV data as DataFrame:")
        print(df.head())

        print("Operation completed successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
df.shape
df.head()

In [ ]:
df_yelp_restaurants = pd.read_csv("restaurants.csv")
df_yelp_restaurants.head()
df_yelp_restaurants.shape

(261031, 27)

In [ ]:
# Data Cleaning
# remove NA from DBA
df_yelp_restaurants_cleaned = df_yelp_restaurants.dropna(subset=['DBA'])
df_yelp_restaurants_cleaned = df_yelp_restaurants_cleaned.dropna(subset=['Longitude'])
df_yelp_restaurants_cleaned = df_yelp_restaurants_cleaned.dropna(subset=['Latitude'])
df_yelp_restaurants_unique = df_yelp_restaurants_cleaned.drop_duplicates(subset=['DBA'], keep='first')
print(df_yelp_restaurants_unique.shape)
df_yelp_restaurants_unique.head()

(23747, 27)


,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location Point1
1,50156071,BAR GIACOSA CORP,Manhattan,268,6 AVENUE,10014.0,2129823300,NaN,01/01/1900,NaN,...,NaN,40.729323,-74.002356,102.0,3.0,6500.0,1008297.0,1.005260e+09,MN23,NaN
2,50083319,SHIMA SUSHI,Manhattan,226,EAST 51 STREET,10022.0,2128298000,Asian/Asian Fusion,08/30/2024,Establishment re-opened by DOHMH.,...,Cycle Inspection / Reopening Inspection,40.756121,-73.969565,106.0,4.0,9800.0,1038450.0,1.013240e+09,MN19,NaN
3,50155587,SALAD HOUSE OF BROOKLYN,Brooklyn,2732,OCEAN AVENUE,11229.0,7188657014,NaN,01/01/1900,NaN,...,NaN,40.594511,-73.950497,315.0,48.0,59200.0,3202114.0,3.074040e+09,BK17,NaN
4,41425803,DONUT SHOPPE,Brooklyn,1503,AVENUE U,11229.0,3476120907,American,08/16/2023,Violations were cited in the following area(s).,...,Cycle Inspection / Re-inspection,40.598852,-73.956263,315.0,48.0,58000.0,3198126.0,3.073200e+09,BK44,NaN
5,50142097,SUM YUNG GAI,Manhattan,17,EAST 13 STREET,10003.0,3477614434,NaN,01/01/1900,NaN,...,NaN,40.734922,-73.993187,102.0,2.0,6100.0,1009401.0,1.005710e+09,MN23,NaN


In [ ]:
staten_island_df = df_yelp_restaurants_unique[df_yelp_restaurants_unique['BORO'] == 'Staten Island']
staten_island_df = staten_island_df.sort_values(by=['Latitude', 'Longitude'])
print(staten_island_df.shape)
bronx_df = df_yelp_restaurants_unique[df_yelp_restaurants_unique['BORO'] == 'Bronx']
bronx_df = bronx_df.sort_values(by=['Latitude', 'Longitude'])
print(bronx_df.shape)
brooklyn_df = df_yelp_restaurants_unique[df_yelp_restaurants_unique['BORO'] == 'Brooklyn']
brooklyn_df = brooklyn_df.sort_values(by=['Latitude', 'Longitude'])
print(brooklyn_df.shape)
manhattan_df = df_yelp_restaurants_unique[df_yelp_restaurants_unique['BORO'] == 'Manhattan']
manhattan_df = manhattan_df.sort_values(by=['Latitude', 'Longitude'])
print(manhattan_df.shape)
queens_df = df_yelp_restaurants_unique[df_yelp_restaurants_unique['BORO'] == 'Queens']
queens_df = queens_df.sort_values(by=['Latitude', 'Longitude'])
print(queens_df.shape)

(829, 27)
(1942, 27)
(6306, 27)
(9109, 27)
(5561, 27)


In [ ]:
# Connect to MongoDB (update the connection string as necessary)
uri = ""

# Create a new client and connect to the server
client = MongoClient(uri, connect=False)
db = client['yelpdata']
collection = db['yelpratings']

In [ ]:
# Create an index on the 'id' to prevent duplicate record
# index_id = collection.create_index([('id', 1)], unique=True)
# print(f"Index created: {index_id}")

# Create an index on the 'name' field
# index_name = collection.create_index([('name', 1)])  # '1' for ascending order
# print(f"Index created: {index_name}")

In [ ]:
!curl ipecho.net/plain

34.16.149.88

In [ ]:
#convert dataframe to json array and send it to the mongodb only run this once.
#json = df_yelp_restaurants.to_dict(orient='records')
#result = collection.insert_many(json)

In [ ]:
# Mock function simulating the API call, returning multiple restaurants
def query_api(Latitude, Longitude, TOKEN):
  headers = { "accept": "application/json", "Authorization": f"Bearer {TOKEN}"}
  url = f"https://api.yelp.com/v3/businesses/search?location=NYC&latitude={Latitude}&longitude={Longitude}&term=restaurants&radius=100&sort_by=review_count&limit=50&offset=1"
  time.sleep(1)
  response = requests.get(url, headers=headers)
  if response.status_code == 200:
    data = response.json()
    business_names = [business['name'] for business in data['businesses']]
    print(business_names)
    return data['businesses'], business_names
  else:
    print(f"Error: {response.status_code}")
    return [], []

In [ ]:
index_to_skip = []
threshold = 60

restaurants_unique_api = list(collection.find({}, {'name': 1, 'rating': 1, 'review_count': 1}))
len(restaurants_unique_api)

NameError: name 'collection' is not defined

In [ ]:
api_calls_count = 0
TOKENS = ["ZJVxqajQcuszlM6kIX__eFij8tyYGchojkX-EvmmwLKqyhn0MzzIcW8rL8PactobeddU74TxGE9RO5cvMX1JMlQMzmsV8UDTLypR0dlVPB1DWCDh_0BNR2qUlt4FZnYx",
          "aty20bBqlAgXn9IpXJczFMQ2oYCumgO_XLCbadkrUUqom92lxZRjAVOcn473qRRrt_VYhTcINyU2GupQbTYJprxfnGocxutF0OUNryeKQJeZ-2lX6CkbJ-0pcZhAZ3Yx",
          "nsyRisz31sLvbSC9yvw6VVpznQECXyrsJRJ0dhycsIJ_cgfO5oHtoFJQGLWjGRe7XVsoOxlJ9A1ypkWa3gZXPxoolYZs20CVGgq-tBQeapgnpfK4pYkAluIXfiJBZ3Yx",
          "d6ouYev36UccN2cyML-Lxf6pkDcLHwImDctX_Bc0mXwP5Csr-I-KUeNA_EF3HQhYhsDMwllnXTKykH6vZYyX2iNjgcpYBj3X-vUQFPqyNS8vl9ZO0UsBPKtsRj9BZ3Yx",
          "q6nPjLSHwRO7XCeBfZCVRx4GyKWTnBo9qE97ukXVYdUiHcAeY5wvfLB9Vy9eM6J9yPe5fnBcKfWYeQ7Cu1KP60cUzB2gmS4-DBnds6fS9tpopsNLiiAkezbbl0FBZ3Yx",
          "9TCXJLCYVMR4dkF5jGokDCAD0-K8i_9GwjtTU8vT_CS2HXp42-d_-IRKr-VbG-grmTv4JdDvo48HMzXurW9lkZ8NCvP76Nm-iGujGDzCWkXoPB8VLPd03dQXK0JBZ3Yx",
          "Iv8C1APe4chPBCDR2NmBha5y8OkX8dgrydgydtg5Wm1Et8vgOfKeE-oTM9MLECCeNOC3lr94e3F8fr2bh1LIcd3rWcunKfVmaqjl_76eJ0hQ10swmt_uNW-Q5UNBZ3Yx",
          "UJhxrz6FXtnH5rdJlPBRZ5FHRYR3bqnoEmM9jp-Hyund_EExo2v3AuOHBsTIFtHaRZV6xmlxgeQAMn4Dy7MHx1VfbjyCaMMIx3wC6rDmmK6uSCTptmOG_HmZEkVBZ3Yx",
          "FT2oKkwTPZxRITVlbwdq7qP8RBQyMR12vk6L51GNOVxYoWWvTBCqYnwTmUnevwQWJ2L0ARvEZrCu6CDdqQC6Wfz_FC3aMudtNmkJvG_TpLjHPl5GjIPFDISNs0VBZ3Yx",
          "gDfTGkXeu17_6n-tRT87oIbjAK6eKALrN4fLSnvME1Ge30YHc6oadDctaknc5K-nEDBnXc5FHJgzS5DvsFJ2HLutYyKPrnfdPCc8oMPGUEerfVpB1bpgzuPzpkZBZ3Yx",
          "g7oZIplhowjeOucpF4ljPuvlnCWolI-HAuNkLA30gd86zKNGl8mLomHV5Rp4hkhCDm_wFMgVJ9cF58nEzQRLXCxdXH90K_c1K2t7JC-qS_1zcJyBJI8_CllpikhBZ3Yx",
          "meI7-mebCZd2qKc_rlx-5ABfUlbPMjHDM_KGdRz3A2wF-PZlfqdalFpvc_xlQasRAv5e7PDlQMVdfgfWv5GiVRvE2EIWEnWC3KHyMQnJ1fn2aQri0LX8pGS0DUpBZ3Yx",
          "OY4nYqGI9tHmzVWyu_Ru6hv6UOizwLDc-tSnj_M_hv5DQI2uLdMq8glTnAC_rv8EOUkj-QgpguiEjFBTХЗUxQarW6PyXE0SGbmE94-SWMZgdAhSoUDW2CfIDbFCZ3Yx",
          "Q66UcGYjJWA0Yj5BoNNxoRyQB3Nfxl7Ss5o7_EQ8LxbJi3l_QP2H05uTdnV5gFwDzjk3wVabY_Oxsm6gFV6LljsU29k0kS_YAly80P5Yzl-Xjp3H4kKfL9KBTIxCZ3Yx"
          ]
token_count = 0

In [ ]:
# Initializing the token_count to retrieve the token to use in the next API CALL
matching_existing = False
# Use iterrows() to iterate through DataFrame rows
for index, row in manhattan_df.iterrows():
    if index in index_to_skip and len(index_to_skip) > 0 :
      continue
    # Check if the current restaurantName is in the set of restaurants to remove
    restaurant_name = row['DBA']
    print(restaurant_name)
    if len(restaurants_unique_api) > 0:
      for restaurant in restaurants_unique_api:
        # Check against each name in the DataFrame
        match_score = fuzz.token_set_ratio(restaurant_name.upper(), restaurant['name'].upper())
        print(f"Match Score For {restaurant_name} with index {index} is {match_score}")
        if match_score >= threshold:
          print(f"{restaurant_name} is in mongo")
          matching_existing = True
          manhattan_df.loc[index, 'yelp_rating'] = restaurant['rating']
          manhattan_df.loc[index, 'yelp_review_count'] = restaurant['review_count']
          break
      # checking if there is a match
      if(matching_existing):
        matching_existing = False
        continue

    # Call the API for each row's latitude and longitude not in restaurants_unique_api

    if(row['Latitude'] == 0):
      continue
    if(row['Longitude'] == 0):
      continue
    api_response_restaurants, restaurant_list = query_api(row['Latitude'], row['Longitude'],TOKENS[token_count])
    api_calls_count += 1

    #Code that switching in the API TOKEN
    print(f"API calls made: {api_calls_count}")
    if(api_calls_count == 300):
      print("API calls limit reached")
      if(token_count <= len(TOKENS) - 1):
        token_count += 1
        api_calls_count = 0

    if(len(api_response_restaurants) > 0):
      # Send to api_response_restaurants to mongodb or anything because I want to keep those values for future references
      # Also making sure that I am skipping any duplicate
      try:
        result = collection.insert_many(api_response_restaurants, ordered=False)
        print("Inserted IDs:", result.inserted_ids)
      except BulkWriteError as e:
        print("Bulk write error occurred:", e.details)

      for restaurant in api_response_restaurants:
        match_score = fuzz.token_set_ratio(restaurant_name.upper(), restaurant['name'].upper())
        if match_score >= threshold:
          # Append the matching restaurant's rating and review count
          print(f"Rating for '{restaurant_name}': {restaurant['rating']}")
          print(f"Review Count for '{restaurant_name}': {restaurant['rating']}")
          manhattan_df.loc[index, 'yelp_rating'] = restaurant['rating']
          manhattan_df.loc[index, 'yelp_review_count'] = restaurant['review_count']
          break
    index_to_skip.append(index)
    restaurants_unique_api = list(collection.find({}, {'name': 1, 'rating': 1, 'review_count': 1}))

In [ ]:
print(TOKENS[token_count])
print(api_calls_count)
print(token_count)

ZJVxqajQcuszlM6kIX__eFij8tyYGchojkX-EvmmwLKqyhn0MzzIcW8rL8PactobeddU74TxGE9RO5cvMX1JMlQMzmsV8UDTLypR0dlVPB1DWCDh_0BNR2qUlt4FZnYx
252
0


In [2]:
staten_island_df.to_csv('staten_island_df.csv', index=False)
print(f"staten_island_df{queens_df.shape}")

brooklyn_df.to_csv('brooklyn_df.csv', index=False)
print(f"brooklyn_df {brooklyn_df.shape}")

queens_df.to_csv('queens_df.csv', index=False)
print(f"queens_df {queens_df.shape}")

bronx_df.to_csv('bronx_df.csv', index=False)
print(f"bronx_df {bronx_df.shape}")

manhattan_df.to_csv('manhattan_df.csv', index=False)
print(f"manhattan_df {manhattan_df.shape}")


SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-2-fe3bbc2de03e>, line 2)

In [ ]:
import json
from bson import ObjectId

class JSONEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, ObjectId):
            return str(o)
        return json.JSONEncoder.default(self, o)

# print api_response_restaurants to json file
restaurants_unique_api_json = json.dumps(restaurants_unique_api, cls=JSONEncoder)
with open('api_response_restaurants.json', 'w') as f:
  f.write(restaurants_unique_api_json)